# Chain-of-Thought Prompting

## Just a quick definition

Chain-of-thought prompting is a technique in prompt engineering where the input prompt encourages the model to reason step-by-step, breaking down complex problems into intermediate steps to improve accuracy and transparency of its outputs.

In [1]:
import os
from openai import OpenAI
from IPython.display import display, Markdown
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if OPENAI_API_KEY is None:
    raise ValueError("Please set the OPENAI_API_KEY environment variable.")

In [2]:
# define response function

client = OpenAI()

def get_response(prompt: str, model: str="gpt-5-chat-latest") -> str:
    response = client.responses.create(
        model=model,
        input=[
            {"role": "user","content": prompt}
        ]
    )
    return response.output_text


In [3]:
# Class to define prompt templates
class PromptTemplate:
    def __init__(self, template: str, input_variables: list[str] | str):
        self.template = template
        self.input_variables = input_variables

    def generate(self, **kwargs) -> str:
        return self.template.format(**{k: kwargs[k] for k in self.input_variables})

In [4]:
# basic template
basic_template = PromptTemplate(
    template="Answer the following question concisely: {question}",
    input_variables=["question"]
)

In [5]:
# chain-of-thought template
cot_template = PromptTemplate(
    template="Answer the following question step-by-step in a clear and concise manner: {question}",
    input_variables=["question"]
)

### 1. Basic Chain-of-Thought Prompting

In [6]:
# question
question = "If a train travels towards a city at 100 km/h, how long will it take to reach the city if it is 150 km away?"

In [7]:
# answer using basic template
print(get_response(basic_template.generate(question=question)))

Time = Distance ÷ Speed = 150 km ÷ 100 km/h = **1.5 hours** (or 1 hour 30 minutes).


In [8]:
# answer using cot template
print(get_response(cot_template.generate(question=question)))

Let's go step-by-step:

1. **Given information:**
   - Speed of the train = 100 km/h  
   - Distance to the city = 150 km  

2. **Formula for time:**
   \[
   \text{Time} = \frac{\text{Distance}}{\text{Speed}}
   \]

3. **Substitute the given values:**
   \[
   \text{Time} = \frac{150\ \text{km}}{100\ \text{km/h}}
   \]

4. **Simplify:**
   \[
   \text{Time} = 1.5\ \text{hours}
   \]

5. **Convert to minutes (optional):**
   \[
   1.5\ \text{hours} = 1\ \text{hour} + 0.5\ \text{hour} = 1\ \text{hour} + 30\ \text{minutes}
   \]

✅ **Final Answer:** 1.5 hours (or 1 hour 30 minutes)


### 2. Writing out clear step-by-step instructions

In [9]:
cot_template = PromptTemplate(
    template="""
    Solve the problem given to you step-by-step in a clear and concise manner.

    Be sure to:
    - Identify the key variables in the question
    - Understand how each variable relates to the problem statement
    - Break down the problem into smaller, manageable steps
    - Solve each step systematically and clearly
    - Give the final answer

    Question:
    {question}
""",
input_variables=["question"]
)

In [10]:
# complex mathematical word problem
question = "A farmer has 3 times as many apples as oranges. If he gives away 12 apples, he will have twice as many apples as oranges. How many apples and oranges does he have?"

In [11]:
print(get_response(cot_template.generate(question=question)))

Let's restate the problem carefully and solve it **step-by-step**.

---

### Step 1: Identify the variables
Let:
- \( A \) = number of apples the farmer has
- \( O \) = number of oranges the farmer has

---

### Step 2: Translate the problem into equations

1. **First relationship**:  
   The farmer has 3 times as many apples as oranges.  
   \[
   A = 3O
   \]

2. **Second relationship (after giving away 12 apples)**:  
   If he gives away 12 apples, he will have twice as many apples as oranges.  
   That means:
   \[
   A - 12 = 2O
   \]

---

### Step 3: Solve the system of equations

We have:
\[
A = 3O
\]
and
\[
A - 12 = 2O
\]

Substitute \( A = 3O \) into the second equation:
\[
3O - 12 = 2O
\]

Simplify:
\[
3O - 2O = 12
\]

\[
O = 12
\]

---

### Step 4: Find number of apples

Using \( A = 3O \):
\[
A = 3(12) = 36
\]

---

### Step 5: Verify the solution

- Original: 36 apples, 12 oranges  
  → \( 36 = 3 \times 12 \) ✅  
- After giving away 12 apples: 24 apples, 12 oranges  
  → 